# LangChain Memory Patterns with Amazon Nova

This notebook demonstrates modern memory patterns for maintaining conversation context with Nova models.

**For current recommendations, see:**
- [Short-term memory](https://python.langchain.com/docs/how_to/chatbots_memory/)
- [Long-term memory](https://python.langchain.com/docs/how_to/chatbots_long_term_memory/)

## Setup

Make sure you have set your environment variables:
```bash
export NOVA_API_KEY=your-api-key
export NOVA_BASE_URL="https://api.nova.amazon.com/v1"
```

In [ ]:
%env NOVA_API_KEY=<YOUR-API-KEY>
%env NOVA_BASE_URL=https://api.nova.amazon.com/v1

In [ ]:
%pip install -e .

In [ ]:
from langchain_amazon_nova import ChatAmazonNova
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# Initialize the model
llm = ChatAmazonNova(
    model="nova-pro-v1",
    temperature=0.7,
    max_tokens=500,
)

## 1. Manual Message History

The simplest approach: manually manage a list of messages.

In [ ]:
# Start with system message
messages = [
    SystemMessage(content="You are a helpful assistant that remembers context.")
]

# Turn 1: User introduces themselves
messages.append(HumanMessage(content="Hi! My name is Alice and I'm learning Python."))
response1 = llm.invoke(messages)
messages.append(AIMessage(content=response1.content))

print("Turn 1:")
print(f"User: Hi! My name is Alice and I'm learning Python.")
print(f"Assistant: {response1.content}\n")

In [ ]:
# Turn 2: Ask about their name (tests memory)
messages.append(HumanMessage(content="What's my name and what am I learning?"))
response2 = llm.invoke(messages)
messages.append(AIMessage(content=response2.content))

print("Turn 2:")
print(f"User: What's my name and what am I learning?")
print(f"Assistant: {response2.content}\n")

In [ ]:
# Check message history
print(f"Total messages in history: {len(messages)}")
for i, msg in enumerate(messages):
    print(f"{i + 1}. {msg.type}: {msg.content[:60]}...")

## 2. RunnableWithMessageHistory (Recommended)

Modern approach using `RunnableWithMessageHistory` for session-based conversations.

In [ ]:
# Create session store
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [ ]:
# First conversation
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi, I'm Bob from Seattle")]},
    config={"configurable": {"session_id": "user_123"}},
)

print(f"User: Hi, I'm Bob from Seattle")
print(f"Assistant: {response.content}\n")

In [ ]:
# Second conversation - remembers context
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What's my name and where am I from?")]},
    config={"configurable": {"session_id": "user_123"}},
)

print(f"User: What's my name and where am I from?")
print(f"Assistant: {response.content}\n")

# Check session history
history = get_session_history("user_123")
print(f"Session has {len(history.messages)} messages")

## 3. Message Trimming (Context Window Management)

Use `trim_messages` to manage the context window and stay within token limits.

In [ ]:
messages = [SystemMessage(content="You are a math tutor. Be concise.")]

# Simulate multiple questions
questions = [
    "What is 15 + 27?",
    "What is 8 * 9?",
    "What is 100 / 4?",
    "What is 12 squared?",
    "What is the square root of 144?",
    "What is 50 - 18?",
]

for i, question in enumerate(questions, 1):
    messages.append(HumanMessage(content=question))
    response = llm.invoke(messages)
    messages.append(AIMessage(content=response.content))
    print(f"Q{i}: {question} -> A{i}: {response.content}")

print(f"\nTotal messages: {len(messages)}")

In [ ]:
# Trim to keep recent conversation
trimmed_messages = trim_messages(
    messages,
    max_tokens=100,
    strategy="last",
    token_counter=len,  # Simple counter for demo
    include_system=True,
    allow_partial=False,
)

print(f"Original: {len(messages)} messages")
print(f"Trimmed: {len(trimmed_messages)} messages")
print(f"\nTrimmed messages:")
for msg in trimmed_messages:
    print(f"  {msg.type}: {msg.content[:50]}...")

## 4. Conversation Summarization (Long-term Memory)

For long conversations, periodically summarize older messages to save tokens.

In [ ]:
# Simulate a longer conversation
conversation_messages = []

conversation_pairs = [
    ("What are quantum computers?", "Quantum computers use quantum bits or qubits that can be in superposition..."),
    ("How do they differ from classical computers?", "Classical computers use binary bits while quantum computers leverage quantum mechanics..."),
    ("What are practical applications?", "Applications include cryptography, drug discovery, optimization problems..."),
    ("Are they commercially available?", "Yes, companies like IBM and Google offer cloud access to quantum computers..."),
]

for user_msg, ai_msg in conversation_pairs:
    conversation_messages.append(HumanMessage(content=user_msg))
    conversation_messages.append(AIMessage(content=ai_msg))

print(f"Conversation has {len(conversation_messages)} messages")

In [ ]:
# Create a summary
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "Provide a concise summary of the conversation below."),
    MessagesPlaceholder(variable_name="messages"),
])

summary_chain = summary_prompt | llm
summary_response = summary_chain.invoke({"messages": conversation_messages})

print("\nConversation Summary:")
print(summary_response.content)
print(f"\nOriginal: ~{sum(len(msg.content) for msg in conversation_messages)} characters")
print(f"Summary: ~{len(summary_response.content)} characters")

## 5. Sliding Window Pattern

Keep only the N most recent exchanges for predictable memory usage.

In [ ]:
class SlidingWindowMemory:
    """Memory that keeps only the N most recent exchanges."""

    def __init__(self, max_exchanges=3):
        self.max_exchanges = max_exchanges
        self.messages = []

    def add_exchange(self, human_msg: str, ai_msg: str):
        self.messages.append(HumanMessage(content=human_msg))
        self.messages.append(AIMessage(content=ai_msg))

        # Keep only last N exchanges (2 messages per exchange)
        max_msgs = self.max_exchanges * 2
        if len(self.messages) > max_msgs:
            self.messages = self.messages[-max_msgs:]

    def get_messages(self):
        return self.messages

    def clear(self):
        self.messages = []


# Test it
sliding_memory = SlidingWindowMemory(max_exchanges=3)

for i in range(5):
    sliding_memory.add_exchange(
        f"This is question number {i + 1}", f"This is answer number {i + 1}"
    )

print(
    f"After 5 exchanges, memory contains {len(sliding_memory.get_messages())} messages:"
)
for msg in sliding_memory.get_messages():
    print(f"  {msg.type}: {msg.content}")

## Summary

**Memory Pattern Comparison:**

| Pattern | Best For | Pros | Cons |
|---------|----------|------|------|
| Manual Message History | Simple use cases | Full control, straightforward | Manual management |
| RunnableWithMessageHistory | Session-based apps | Automatic session management | Needs storage backend |
| Message Trimming | Token management | Predictable usage | May lose context |
| Summarization | Long conversations | Bounded memory | Extra LLM calls, loses detail |
| Sliding Window | Production chatbots | Consistent performance | Limited history |

**Recommended Approaches:**
- **Short conversations**: Manual history or RunnableWithMessageHistory
- **Long sessions**: Combine trimming + summarization
- **Production**: RunnableWithMessageHistory with persistent storage

For more details, see the [LangChain memory documentation](https://python.langchain.com/docs/how_to/chatbots_memory/).